In [ ]:
import numpy as np
import random
from math import *
import cmath
from scipy.linalg import expm, fractional_matrix_power

#########################################################################

# pauli matrix 
sx = np.array([[0,  1],     [1, 0]])
sy = np.array([[0, -1j],   [1j, 0]])
sz = np.array([[1, 0],     [0, -1]])
s0 = np.array([[1, 0],      [0, 1]])

# parameters(detuning factor)
v0 = 0.02           # Arbitrary settings, Actual speed : 0.04rad/μs
d0 = 0.15           # Arbitrary settings, Actual speed : 0.30rad/μs
dt = 2.6 


# unitary operator
def unitary(dt, choice):
    
    # Select x,y-rotation direction.
    # [stay, +x, -x, +y, -y]
    choice_list = [0, 1, -1, 1, -1] 
    
    if choice < 3:
        # if choice = 0 ... only d0*sz
        Ham = (d0*sz+v0*choice_list[choice]*sx)
    else:
        Ham = (d0*sz+v0*choice_list[choice]*sy)

    # Creating a Unitary Operator for each of the four sections by Hamiltonian
    eigvals = np.linalg.eigh(Ham)[0]
    eigvecs = 1*np.linalg.eigh(Ham)[1]
    E = np.diag(eigvals)
    U_H = eigvecs.conj().T
    U_e = U_H.conj().T @ expm(-1j*E*dt) @ U_H
    
    return U_e


#########################################################################

# x-rotation operater
def Rx(theta):
    return np.matrix([  [cos(theta/2),    -1j*sin(theta/2)],
                        [-1j*sin(theta/2),    cos(theta/2)]])

# z-rotation operater
# Do not use Rz. Control by rotation only by Hamiltonian.
def Rz(phi): 
    return np.matrix([  [cos(phi/2)-1j*sin(phi/2),  0],
                        [0,  cos(phi/2)+1j*sin(phi/2)]])

# Calculating the Fidelity
def state_fidelity(rho_1, rho_2): 
    
    # rho_1(current state), rho_2(target state)
    # Calculate the fidelity after checking the dimensions of the two states.
    
    if np.shape(rho_1) != np.shape(rho_2):
        print("Dimensions of two states do not match.")
        return 0
    else:
        sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
        fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
        return np.real(fidelity)


#########################################################################

init_wave = np.array([[1], [0]])
irho_init = np.kron(init_wave,init_wave.conj().T)

U_0 = unitary(dt, 0)
U_1 = unitary(dt, 1)
U_2 = unitary(dt, 2)
U_3 = unitary(dt, 3)
U_4 = unitary(dt, 4)

pulse_list = [U_0, U_1, U_2, U_3, U_4]


def fidelity(rho, predicted_sequence):

    # predicted_sequence_indices = np.argmax(predicted_sequence, axis=-1)

    Uni = s0
    
    for i in predicted_sequence:
        Uni = pulse_list[i] @ Uni

    irho_final = Uni @ irho_init @ Uni.conj().T
    
    F = (state_fidelity(irho_final, rho))
    
    return  F

In [ ]:
def find_best_seq(seq):

  # 리스트의 요소를 순서대로 탐색합니다.
  for i in range(1, len(seq) + 1):
    for j in range(5):
      new_seq = seq[:i] + [j] + seq[i + 1:]
      fid = fidelity(new_seq)

      # 현재 리스트보다 fidelity가 더 높으면, 최종 리스트로 선택합니다.
      if fid > fidelity(seq):
        seq = new_seq

  return seq

In [ ]:
import time

input_theta = 0
input_phi = 0

target_U = Rz(input_phi) @ Rx(input_theta)
input_rho = target_U @ irho_init @ target_U.conj().T

input_sequence = []
output_sequence = []

if fidelity(input_rho, input_sequence) > 99.99:
     output_sequence = input_sequence
else:
     start_time = time.time()
     
     output_sequence = find_best_seq(input_sequence)
     
     end_time = time.time()
     computing_time = end_time - start_time

total_time = dt * len(output_sequence)

# Result output
print(f"""
------------------------------------------------------------------------------------------------------
theta = {input_theta}
phi = {input_phi}
input_sequence : {input_sequence}
output_sequence : {output_sequence}
computing_time : {computing_time}
total_time : {total_time}
------------------------------------------------------------------------------------------------------
""")